In [1]:
! pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 KB 20.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging
import sklearn

covid_train_path = '/content/drive/My Drive/covidworry_train.csv'
covid_test_path = '/content/drive/My Drive/covidworry_test.csv'

covid_train = pd.read_csv(covid_train_path)
covid_test = pd.read_csv(covid_test_path)

covidworry_edaaug_5_path = '/content/drive/My Drive/covidworry_edaaug_5.csv'
covid_data = pd.read_csv(covidworry_edaaug_5_path, encoding='utf-8')


print(covid_train.shape)
print(covid_data.shape)

(1685, 2)
(8425, 3)


Concatenate train and praphased data




In [5]:
parafrase = covid_data[['paraphrase', 'label']].copy()
parafrase = parafrase.drop_duplicates()
parafrase.rename(columns={"paraphrase": "essay","label":"emotion"}, inplace = True)

covid_train = covid_train.append(parafrase,ignore_index=True)
print(covid_train.shape)

(10110, 2)


<ipython-input-5-758050b749b5>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  covid_train = covid_train.append(parafrase,ignore_index=True)


In [6]:
train_df = pd.DataFrame(covid_train)
train_df.columns = ["text", "labels"]

eval_df = pd.DataFrame(covid_test)
eval_df.columns = ["text", "labels"]

In [7]:
import torch
from torch import cuda
import gc

def empty_cache(model, optimizer):
  device = 'cuda' if cuda.is_available() else 'cpu'
  with torch.no_grad():
      torch.cuda.empty_cache()
  gc.collect()

In [8]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, precision_score, recall_score

def f1_multiclass(labels, preds):
  return f1_score(labels, preds, average='macro')

def precision_multiclass(labels,preds):
  return precision_score(labels, preds, average='macro')

def recall_multiclass(labels,preds):
  return recall_score(labels, preds, average='macro')


In [9]:
results = []

reps = 3
for i in range(reps):

  model = ClassificationModel(
    "roberta", 
    "roberta-base",
    num_labels=5, 
    args={'num_train_epochs':2,
          'max_seq_length':256,
          'learning_rate':1e-5,
          'overwrite_output_dir': True, 
          'custom_callback': empty_cache
          }
  )

  model.train_model(train_df)

  result, model_outputs, wrong_predictions = model.eval_model(eval_df, 
                                                              acc=sklearn.metrics.accuracy_score,
                                                              f1=f1_multiclass,
                                                              precision=precision_multiclass,
                                                              recall=recall_multiclass,
                                                              )

  
  results.append({'accuracy': result['acc'], 'f1': result['f1'], 'precision': result['precision'], 'recall': result['recall']})

print('accuracy, f1, precision, recall:')
for i in results:
  print(i['accuracy'], i['f1'], i['precision'], i['recall'])

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

  0%|          | 0/10110 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/1264 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/1264 [00:00<?, ?it/s]

...

Running Epoch 0 of 2:   0%|          | 0/1264 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/1264 [00:00<?, ?it/s]

  0%|          | 0/723 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/91 [00:00<?, ?it/s]

accuracy, f1, precision, recall:
0.5809128630705395 0.44758415468978124 0.4558270616695682 0.4408797587750432
0.5961272475795297 0.4510029406036473 0.4657765661792272 0.43996674548661935
0.5905947441217151 0.46670053627912395 0.4851897480045609 0.45455661328777114
